## Installations

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
!sudo apt-get update && sudo apt-get install ffmpeg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                     
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease      
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Rea

In [8]:
pip install -U datasets[audio] librosa lightning torch torchvision torchaudio torchlibrosa museval torchcontrib protobuf==5.29.3 tqdm --extra-index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 6.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 5.29.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 req

## Data Processing

In [51]:
import pandas as pd
import librosa
from tqdm import tqdm

NON_EVENT_LABEL = 'non_event'

train_df = pd.read_csv("train_data.csv")
train_df.head()


train_info_df = []

for row in tqdm(train_df.itertuples(), desc="Loading Audio Files"):
    train_info_df.append(
        {
            "file": row.file_name,
            "file_path": row.file_path,
            "S_db": None,
            "onset": row.onset,
            "offset": row.offset,
            "event_label": row.label,
            "background_label": None,
            "sr": None
        }
    )

Loading Audio Files: 4564it [00:00, 485512.92it/s]


In [52]:
from src.config import DETECTION_TRAIN_PATH, DETECTION_TEST_PATH
from src.config import CLASSES
import pickle

NON_EVENT_LABEL = 'non_event'

## Train
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
num_records = len(data["files"])
# train_info_df = []
for i in range(num_records):
    train_info_df.append(
        {
            "file": data["files"][i],
            "file_path": f'{DETECTION_TRAIN_PATH}/{data["files"][i]}',
            "S_db": data["S_db"][i],
            "onset": data["onset"][i],
            "offset": data["offset"][i],
            "event_label": data["event_label"][i],
            "background_label": data["background_label"][i],
            "sr": data["sr"]

        }
    )



## Test
data = pickle.load(open('data/processed/yamnet/spectrograms_test.pkl', 'rb'))
num_records = len(data["files"])
test_info_df = []
for i in range(num_records):
    test_info_df.append(
        {
            "file": data["files"][i],
            "file_path": f'{DETECTION_TEST_PATH}/{data["files"][i]}',
            "S_db": data["S_db"][i],
            "onset": data["onset"][i],
            "offset": data["offset"][i],
            "event_label": data["event_label"][i],
            "background_label": data["background_label"][i],
            "sr": data["sr"]

        }
    )

In [ ]:
# Using Base from HTS AT Repository
from external.hts_audio_transformer import config

## Add / Modify Configurations
config.debug = True
config.max_epoch = 10
config.classes_num = len(CLASSES) + 1
config.sample_rate = train_info_df[0]['sr']

In [ ]:
import torch
import torch.nn.functional as F

def ensure_fixed_waveform_length(waveform: torch.Tensor, target_length: int = 882000) -> torch.Tensor:
    """
    Ensures an audio waveform tensor is exactly the target_length by cropping the start 
    or zero-padding the end, using optimized PyTorch operations.

    Args:
        waveform (torch.Tensor): The input audio time series tensor (e.g., shape [L]).
        target_length (int): The required length for the output waveform.

    Returns:
        torch.Tensor: The processed waveform with the exact target_length.
    """
    original_length = waveform.size(0)

    if original_length > target_length:
        return waveform[:target_length]
        
    elif original_length < target_length:
        padding_amount = target_length - original_length
        padding_tuple = (0, padding_amount)

        return F.pad(waveform, padding_tuple, mode='constant', value=0.0)
        
    else:
        return waveform


In [50]:
from datasets import Dataset, ClassLabel
import librosa

CLASS_LABELS = ClassLabel(names=CLASSES + [NON_EVENT_LABEL])

train_info_dataset = Dataset.from_list(train_info_df)
test_info_dataset = Dataset.from_list(test_info_df)

def format_dataset(dataset: Dataset):

    def format_function(record, idx):
        audio_filename= record["file"]

        audio_path = record['file_path']
        audio, sr = librosa.load(audio_path, sr=record['sr'])
        target = CLASS_LABELS.str2int(record['event_label'])

        print(len(audio))

        return record | {
            "audio_name": audio_filename,
            "target": target,
            "waveform": audio,
            "real_len": len(audio)
        }

    formatted_dataset = dataset.map(
        format_function,
        with_indices=True,
    ).with_format("torch")

    return formatted_dataset

formatted_train_dataset = format_dataset(train_info_dataset)
formatted_test_dataset = format_dataset(test_info_dataset)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000
882000

KeyboardInterrupt: 

In [7]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import logging
import random

class HTSATDataset(Dataset):
    def __init__(self, dataset, config, eval_mode = False):
        self.dataset = dataset
        self.config = config       
        self.total_size = len(self.dataset)
        self.queue = [*range(self.total_size)]
        logging.info("total dataset size: %d" %(self.total_size))
        if not eval_mode:
            self.generate_queue()

    def generate_queue(self):
        random.shuffle(self.queue)
        logging.info("queue regenerated:%s" %(self.queue[-5:]))


    def __getitem__(self, index):
        # Get actual index of the record
        p = self.queue[index]

        # Get actual item
        record = self.dataset[p]

        # one-hot target if required
        if self.config.loss_type == 'clip_bce':
            label_tensor = torch.as_tensor(record["target"], dtype=torch.long)
            target = F.one_hot(label_tensor, num_classes=self.config.classes_num).float()
        else:
            target = torch.as_tensor(record["target"])

        data_dict = record | {
            "target": target
        }
        logging.info(f"getitem: {p}")
        return data_dict

    def __len__(self):
        return self.total_size


htsat_train_dataset = HTSATDataset(formatted_train_dataset, config)
htsat_test_dataset = HTSATDataset(formatted_test_dataset, config)

print(f"Train Dataset: {len(htsat_train_dataset)}")
print(f"Test Dataset: {len(htsat_test_dataset)}")

Train Dataset: 2500
Test Dataset: 500


## Create Model

In [8]:
# del minimal_trainer
# del sed_model
# del model

In [9]:
config.clip_duration = 20.0
config.classes = CLASS_LABELS
config.clip_samples = config.sample_rate * config.clip_duration
config.htsat_spec_size = 512

In [10]:
import lightning as L
import torch.optim as optim
import bisect
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score
import numpy as np

from external.hts_audio_transformer.utils import get_loss_func


class HTSATModel(L.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.config = config
        # self.dataset = dataset
        self.loss_func = get_loss_func(config.loss_type)
        self.test_step_outputs = []
    
    def forward(self, x, mix_lambda = None):
        output_dict = self.model(x)
        return output_dict["clipwise_output"] , output_dict["framewise_output"]
    
    def training_step(self, batch, batch_idx):
        self.device_type = next(self.parameters()).device
        mix_lambda = None

        batch_waveform = batch['waveform']
        pred, pred_map = self(batch_waveform, mix_lambda)

        # Get Frame Loss
        loss = self.calculate_framewise_loss(batch, pred_map)

        # # Get Clip Loss
        # target = batch['target']
        # clip_loss = self.loss_func(pred, target)

        # Combine
        # loss = 0.5 * clip_loss + 0.5 * frame_loss

        self.log("loss", loss, on_epoch= True, prog_bar=True, batch_size=self.config.batch_size)

        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr = self.config.learning_rate, 
            betas = (0.9, 0.999), eps = 1e-08, weight_decay = 0.05, 
        )
        # Change: SWA, deprecated
        # optimizer = SWA(optimizer, swa_start=10, swa_freq=5)
        def lr_foo(epoch):       
            if epoch < 3:
                # warm up lr
                lr_scale = self.config.lr_rate[epoch]
            else:
                # warmup schedule
                lr_pos = int(-1 - bisect.bisect_left(self.config.lr_scheduler_epoch, epoch))
                if lr_pos < -3:
                    lr_scale = max(self.config.lr_rate[0] * (0.98 ** epoch), 0.03 )
                else:
                    lr_scale = self.config.lr_rate[lr_pos]
            return lr_scale
        scheduler = optim.lr_scheduler.LambdaLR(
            optimizer,
            lr_lambda=lr_foo
        )
        
        return [optimizer], [scheduler]
    
    def validation_step(self, batch, batch_idx):
        pred, _ = self(batch["waveform"])
        return [pred.detach(), batch["target"].detach()]
    
    def time_shifting(self, x, shift_len):
        shift_len = int(shift_len)
        new_sample = torch.cat([x[:, shift_len:], x[:, :shift_len]], axis = 1)
        return new_sample 
    
    def test_step(self, batch, batch_idx):
        pred, pred_map = self(batch["waveform"])    # pred: (B, C) | pred_map: (B, window_size, C)
        target = batch['target']                    # target: (B, C)
        onset = batch['onset']
        offset = batch['offset']
        loss = self.loss_func(pred, target)

        output = {
            'file': batch['file'],
            'event_label': batch['event_label'],
            'pred': pred.detach().cpu(), 
            'pred_map': pred_map.detach().cpu(),
            'target': target.detach().cpu(),
            'onset': onset.detach().cpu(),
            'offset': offset.detach().cpu(),
        }
        self.test_step_outputs.append(output)

        return
    
    # def evaluate_metric(self, pred, target):
        # # Convert to Numpy for use
        # pred_numpy = pred.numpy()
        # target_numpy = target.numpy()

        # # Accuracy
        # predicted_labels = np.argmax(pred_numpy, axis=1)
        # true_labels = np.argmax(target_numpy, axis=1)
        # acc = accuracy_score(true_labels, predicted_labels)

        # # Loss
        # loss = self.loss_func(pred, target).item()

        # # mAP
        # mAP = average_precision_score(target_numpy, pred_numpy, average='macro')

        # return {
        #     "acc": acc,
        #     "loss": loss,
        #     "mAP": mAP
        # }

    def on_test_epoch_end(self):
        test_step_outputs = self.test_step_outputs

        # all_preds = torch.cat([x['pred'] for x in test_step_outputs], dim=0)
        # all_targets = torch.cat([x['target'] for x in test_step_outputs], dim=0)
    
        # metric_results = self.evaluate_metric(all_preds, all_targets)

        # self.log_dict(metric_results,  on_epoch=True, prog_bar=True, sync_dist=True)
    
    def calculate_framewise_loss(self, batch, pred_map):
        self.device_type = next(self.parameters()).device

        batch_size, num_frames, num_classes = pred_map.shape

        # Frame-Wise Loss Calculation
        # Construct the Frame-wise Target Map
        batch_target = batch['target'].to(self.device_type).float() # (shape: 10, 6)
        batch_onset = batch['onset'].to(self.device_type).float()  # (shape: 10)
        batch_offset = batch['offset'].to(self.device_type).float()   # (shape: 10)

        target_map = torch.zeros(batch_size, num_frames, num_classes, device=self.device_type)
        target_map[:, :, self.config.classes.str2int(NON_EVENT_LABEL)] = 1

        frame_onset = batch_onset / self.config.clip_duration * num_frames   # (shape: 10)
        frame_offset = batch_offset / self.config.clip_duration * num_frames   # (shape: 10)

        frame_onset = frame_onset.round().long()
        frame_offset = frame_offset.round().long()

        onset_broadcast = frame_onset.unsqueeze(1)    # Shape (10, 1)
        offset_broadcast = frame_offset.unsqueeze(1)  # Shape (10, 1)

        frame_indices = torch.arange(num_frames, device=self.device_type)
        mask = (frame_indices >= onset_broadcast) & (frame_indices < offset_broadcast)
        mask_expanded = mask.unsqueeze(-1).expand_as(target_map) # (10, 1024, 6)target_expanded = batch_target.unsqueeze(1) # (10, 1, 6)

        target_expanded = batch_target.unsqueeze(1) # (10, 1, 6)
        target_map = torch.where(mask_expanded, target_expanded, target_map)

        loss = self.loss_func(pred_map, target_map)

        return loss

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = htsat_train_dataset,
    num_workers = config.num_workers,
    # num_workers = 0,
    batch_size = 10,
    shuffle = False,
    # sampler = train_sampler
)


# val_loader = DataLoader(
#     dataset = htsat_val_dataset,
#     num_workers = config.num_workers,
#     # num_workers = 0,
#     batch_size = 10,
#     shuffle = False,
#     # sampler = train_sampler
# )

test_loader = DataLoader(
    dataset = htsat_test_dataset,
    num_workers = config.num_workers,
    # num_workers = 0,
    batch_size = 10,
    shuffle = False,
    # sampler = train_sampler
)

In [12]:
minimal_trainer = L.Trainer(
    # --- Essentials ---
    max_epochs=config.max_epoch,         # Stop condition for training
    default_root_dir="./checkpoints",     # Where logs and checkpoints go

    # --- Minimal Device Configuration (Assuming single GPU/CPU) ---
    # Setting devices=1 for modern Lightning (replaces gpus=1)
    # If device_num is 0 or 1, you can often just omit this
    # to let Lightning detect the device, but specifying is safer.
    # Note: If you need to specify CPU/GPU, use: devices=1, accelerator='gpu'/'cpu'
    
    # Keeping 'gpus' for compatibility with older PL versions in your original code:
    # gpus=device_num, 
    # log_every_n_steps = 1
)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


In [13]:
from external.hts_audio_transformer.model.htsat import HTSAT_Swin_Transformer

sed_model = HTSAT_Swin_Transformer(
    spec_size=512,
    num_classes=config.classes_num,
    config = config
)
model = HTSATModel(sed_model, config) 

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
minimal_trainer.fit(model, train_dataloaders=train_loader)

/usr/local/lib/python3.11/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2025-12-02 16:53:29.707652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764694409.888943    5155 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764694409.936774    5155 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register fact

┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name  ┃ Type                   ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model │ HTSAT_Swin_Transformer │ 28.6 M │ train │     0 │
└───┴───────┴────────────────────────┴────────┴───────┴───────┘

Trainable params: 27.6 M                                                                                           
Non-trainable params: 1.1 M                                                                                        
Total params: 28.6 M                                                                                               
Total estimated model params size (MB): 114                                                                        
Modules in train mode: 217                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [39]:
# trained_model = model.model
# model = HTSATModel(trained_model, config)
test_results = minimal_trainer.test(
    model, 
    dataloaders=test_loader
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

In [ ]:
from scipy.ndimage import gaussian_filter1d

def get_events(batch, config):
    NON_EVENT_LABEL_INT = CLASS_LABELS.str2int(NON_EVENT_LABEL)

    file = batch['file']
    pred_map = batch['pred_map']
    batch_size, num_frame, num_classes = pred_map.shape


    # pred_map_median = median_filter(pred_map, size=(1, 7, 1)) # smoothing between frames
    pred_map_gaussian_np = gaussian_filter1d(
        input=pred_map, 
        sigma=3,
        axis=1,             # Apply the filter along the time axis (num_frame)
        mode='nearest'      # How to handle the boundaries of the frames
    )
    pred_map = torch.as_tensor(pred_map_gaussian_np)
    pred_map_labels = torch.argmax(pred_map, dim=2)

    events = {}

    for batch_idx in range(batch_size):
        pred_map_record = pred_map_labels[batch_idx]
        file_record = file[batch_idx]
        onset_frame = 0
        file_events = []
        current_label = pred_map_record[0]
        for i, pred_label in enumerate(pred_map_record, start = 1):
            if pred_label != current_label:
                if current_label != NON_EVENT_LABEL_INT:
                    onset_time = onset_frame / num_frame * config.clip_duration
                    offset_time = i / num_frame * config.clip_duration
                    file_events.append(
                        {
                            'file': file_record,
                            'event_onset': onset_time,
                            'event_offset': offset_time,
                            'event_label': CLASS_LABELS.int2str(current_label.item())
                        }
                    )
                    
                onset_frame = i
                current_label = pred_label

        if current_label != NON_EVENT_LABEL_INT:
            onset_time = onset_frame / num_frame * config.clip_duration
            offset_time = i / num_frame * config.clip_duration
            file_events.append(
                {
                    'file': file_record,
                    'event_onset': onset_time,
                    'event_offset': offset_time,
                    'event_label': CLASS_LABELS.int2str(current_label.item())
                }
            )
        events[file_record] = file_events
    return events

pred_event_dict = {}
for batch in model.test_step_outputs:
    batch_events = get_events(batch, config)
    pred_event_dict = pred_event_dict | batch_events

In [45]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils.audio_to_spectrograms import create_spectrogram_pkl

# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'
if not(os.path.exists(gt_pkl_path)):
    create_spectrogram_pkl()

gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [49]:
from loguru import logger
import sed_eval

from src.config import CLASSES

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    # event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES, t_collar=t_collar)
    # for file, estimated_event in pred_event_dict.items():
    #     ref_event = gt_event_dict[file]
    #     event_based_metrics.evaluate(
    #         reference_event_list=ref_event,
    #         estimated_event_list=estimated_event
    #     )
    # print(event_based_metrics)

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(CLASSES, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(segment_based_metrics)

    return event_based_metrics, segment_based_metrics

event_based_metrics, segment_based_metrics = calculate_metrics(pred_event_dict, gt_event_dict, t_collar=0.5)

2025-12-02 17:41:20.956 | WARNING  | __main__:calculate_metrics:7 - Metrics not implemented!


Segment based metrics
  Evaluated length                  : 4130.00 sec
  Evaluated files                   : 500 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 82.27 %
    Precision                       : 84.90 %
    Recall                          : 79.81 %
  Error rate
    Error rate (ER)                 : 0.28 
    Substitution rate               : 0.07 
    Deletion rate                   : 0.13 
    Insertion rate                  : 0.07 
  Accuracy
    Sensitivity                     : 79.81 %
    Specificity                     : 98.51 %
    Balanced accuracy               : 89.16 %
    Accuracy                        : 96.73 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 82.30 %
    Precision                       : 85.61 %
    Recall                          : 80.34 %
  Error rate
    Error rate (ER)                 : 0.34 
    Deletion rat

In [53]:
!mkdir saved_models

In [54]:
import torch

# Assume 'model' is your initialized PyTorch model
# Define the path where you want to save the file
PATH = 'saved_models/htsat_3Dec_10Epoch.pth'

# Use torch.save() to save the state dictionary
trained_model = model.model
torch.save(trained_model.state_dict(), PATH)